# Assignment 2 - Machine Learning
## FEUP MIEIC - Inteligência Artificial *(EIC0029/IART)*



## Neural Network partial notebook

This Jupyter Notebook implpementes and documents a Neural Network Machine Learning model. <br><br>
*The content of this notebook will be also included in the complete notebook named **IART-MachineLearning.ipynb** that can be found in the root directory of this repository*

***

In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Connecting to database
database = "database.sqlite"
con = sqlite3.connect(database)
pd.set_option('display.max_columns', None)

#Get important tables from db
matches_df = pd.read_sql("""SELECT * from MATCH""", con)
teams_df = pd.read_sql("""SELECT * from TEAM""", con)
player_attributes_df = pd.read_sql("""SELECT * from PLAYER_ATTRIBUTES""", con)
team_attributes_df = pd.read_sql("""SELECT * from TEAM_ATTRIBUTES""", con)

#Get players' attributes (overall)

#Get players info
home_players = ["home_player_" + str(x) for x in range(1, 12)]
away_players = ["away_player_" + str(x) for x in range(1, 12)]

matches_kept_columns = ["id", "league_id", "date", "home_team_api_id", "away_team_api_id", "home_team_goal", "away_team_goal"]
matches_kept_columns = matches_kept_columns + home_players
matches_kept_columns = matches_kept_columns + away_players

matches_df = matches_df[matches_kept_columns]

#Get overall ratings for all players from player_attributes table
for player in home_players:
    matches_df = pd.merge(matches_df, player_attributes_df[["id", "overall_rating"]], left_on=[player], right_on=["id"], suffixes=["", "_" + player])
for player in away_players:
    matches_df = pd.merge(matches_df, player_attributes_df[["id", "overall_rating"]], left_on=[player], right_on=["id"], suffixes=["", "_" + player])
 

matches_df = matches_df.rename(columns={"overall_rating": "overall_rating_home_player_1"})

matches_df['overall_rating_home'] = matches_df[['overall_rating_' + p for p in home_players]].sum(axis=1)
matches_df['overall_rating_away'] = matches_df[['overall_rating_' + p for p in away_players]].sum(axis=1)
matches_df['overall_rating_difference'] = matches_df['overall_rating_home'] - matches_df['overall_rating_away']

matches_df['mean_overall_rating_home'] = matches_df[['overall_rating_' + p for p in home_players]].mean(axis=1)
matches_df['mean_overall_rating_away'] = matches_df[['overall_rating_' + p for p in away_players]].mean(axis=1)


#Remove all players column because we just need the "global" ones
for c in matches_df.columns:
    if '_player_' in c:
        matches_df = matches_df.drop(c, axis=1)

#function to calculate last 5 games performance
def last5(team_id, date, match_t):
     
    mat = match_t[(match_t['date'] < date)]
    mat = mat[mat['home_team_api_id'] == team_id]
    
    mat5 = mat.head(5)
    if len(mat5.index) < 5:
        return -1
    mat5['Home result'] = 0
    mat5['Home result'] = np.where(mat5['home_team_goal'] > mat5['away_team_goal'], 3, mat5['Home result'])
    mat5['Home result'] = np.where(mat5['home_team_goal'] == mat5['away_team_goal'], 1, mat5['Home result'])
    total = mat5['Home result'].sum()
    return total

match_t = matches_df[['home_team_api_id', 'away_team_api_id','date', 'home_team_goal', 'away_team_goal']].copy()
match_t['date'] = pd.to_datetime(match_t['date'])
match_t.sort_values(by=['date'], inplace=True, ascending=False) 
matches_df['Home_last5'] = 0
matches_df['Away_last5'] = 0
perc = 0
for i in matches_df.index:
    Htotal = last5(match_t['home_team_api_id'].iloc[i], match_t['date'].iloc[i], match_t)
    Atotal = last5(match_t['away_team_api_id'].iloc[i], match_t['date'].iloc[i], match_t)
    matches_df['Home_last5'].values[i] = Htotal
    matches_df['Away_last5'].values[i] = Atotal
    if i % 655 == 0:
        perc += 10
        print(perc, "%", end=" ")

matches_df

matches_aux = pd.read_sql("""SELECT * FROM MATCH""" ,con)

#Select all bet columns (removed PSA, PSH, PSD because they are almost all NaN)
bet_columns = ["B365H", "B365A", "B365D", "BWH", "BWD", "BWA", "IWH", "IWD", "IWA", "LBH", "LBD", "LBA", "WHH", "WHD", "WHA", "SJH", "SJD", "SJA", "VCH", "VCD", "VCA", "GBH", "GBD", "GBA", "BSH", "BSD", "BSA"]

#Get specific columns for bets on home and draw
bet_columns_home = ["B365H", "BWH","IWH", "LBH", "WHH", "SJH", "VCH", "GBH","BSH"]
bet_columns_draw = ["B365D", "BWD","IWD", "LBD", "WHD", "SJD", "VCD", "GBD","BSD"]

#Calculate mean values for bets on home team and draw. Add these values to match table
matches_df['mean_bets_home'] = matches_aux[bet_columns_home].mean(axis=1)
matches_df['mean_bets_draw'] = matches_aux[bet_columns_draw].mean(axis=1)
#Replace NaN values (on bets) with mean values 
matches_df.fillna(matches_df.mean(), inplace=True)
matches_df.isnull().sum()

#Get goal difference
matches_df['goal_diff'] = matches_df['home_team_goal'] - matches_df['away_team_goal']

matches_df['Game Result'] = 'Defeat'
matches_df['Game Result'] = np.where(matches_df['goal_diff'] == 0, 'Draw', matches_df['Game Result'])
matches_df['Game Result'] = np.where(matches_df['goal_diff'] > 0, 'Win', matches_df['Game Result'])

matches_df

10 %20 %30 %40 %50 %60 %70 %80 %90 %100 %

,id,league_id,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,overall_rating_home,overall_rating_away,overall_rating_difference,mean_overall_rating_home,mean_overall_rating_away,Home_last5,Away_last5,mean_bets_home,mean_bets_draw,goal_diff,Game Result
0,8227,7809,2010-01-30 00:00:00,8177,9911,0,0,746.0,783.0,-37.0,67.818182,71.181818,1,1,1.765556,3.322222,0,Draw
1,8152,7809,2009-11-22 00:00:00,9790,9911,0,1,772.0,790.0,-18.0,70.181818,71.818182,8,11,1.898889,3.250000,-1,Defeat
2,8109,7809,2008-10-26 00:00:00,10269,9911,2,0,780.0,719.0,61.0,70.909091,71.900000,9,8,2.462222,3.233333,2,Win
3,8088,7809,2008-10-04 00:00:00,9823,9911,3,3,688.0,717.0,-29.0,68.800000,71.700000,8,3,1.438889,3.805556,0,Draw
4,8010,7809,2008-08-30 00:00:00,10189,9911,1,0,733.0,770.0,-37.0,66.636364,70.000000,8,-1,4.494444,3.422222,1,Win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6537,20738,19694,2012-08-19 00:00:00,9938,8284,3,0,772.0,780.0,-8.0,70.181818,70.909091,-1,-1,2.044444,3.144444,3,Win
6538,20577,19694,2011-08-20 00:00:00,9938,8457,0,1,755.0,720.0,35.0,68.636364,65.454545,-1,-1,2.284444,3.097778,-1,Defeat
6539,17958,17642,2010-02-07 00:00:00,7841,6421,2,0,752.0,779.0,-27.0,68.363636,70.818182,-1,-1,2.282222,3.066667,2,Win
6540,13834,13274,2009-08-23 00:00:00,8611,10219,2,1,682.0,755.0,-73.0,68.200000,68.636364,-1,-1,2.515556,3.100000,1,Win


In [13]:
matches_df = matches_df.drop(['id', 'date', 'home_team_api_id','away_team_api_id','home_team_goal','away_team_goal','overall_rating_difference', 'goal_diff'], axis=1)
matches_df


,league_id,overall_rating_home,overall_rating_away,mean_overall_rating_home,mean_overall_rating_away,Home_last5,Away_last5,mean_bets_home,mean_bets_draw,Game Result
0,7809,746.0,783.0,67.818182,71.181818,1,1,1.765556,3.322222,Draw
1,7809,772.0,790.0,70.181818,71.818182,8,11,1.898889,3.250000,Defeat
2,7809,780.0,719.0,70.909091,71.900000,9,8,2.462222,3.233333,Win
3,7809,688.0,717.0,68.800000,71.700000,8,3,1.438889,3.805556,Draw
4,7809,733.0,770.0,66.636364,70.000000,8,-1,4.494444,3.422222,Win
...,...,...,...,...,...,...,...,...,...,...
6537,19694,772.0,780.0,70.181818,70.909091,-1,-1,2.044444,3.144444,Win
6538,19694,755.0,720.0,68.636364,65.454545,-1,-1,2.284444,3.097778,Defeat
6539,17642,752.0,779.0,68.363636,70.818182,-1,-1,2.282222,3.066667,Win
6540,13274,682.0,755.0,68.200000,68.636364,-1,-1,2.515556,3.100000,Win


In [30]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier



y = matches_df['Game Result']
X = matches_df.drop('Game Result', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42, 
                                                    stratify=y)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                         hidden_layer_sizes=(15, 15,15, 15,15, 15),verbose=10, random_state=1, tol=1e-6)
clf.fit(X_train_scaled, y_train)
x=clf.predict(scaler.transform(X_test))
np.set_printoptions(threshold=sys.maxsize)


from sklearn.metrics import accuracy_score

accuracy_score(y_test, x)

0.40295466123280693

In [33]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

parameters = {'hidden_layer_sizes':((10,3), (15,10))}
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,verbose=10, random_state=1, tol=1e-6);
clf = GridSearchCV(mlp, parameters)

